In [1]:
import pickle
import dask.dataframe as dd
import pandas as pd

In [7]:
eICU_features = set(pd.read_csv('../1_importsql/feature_df2.csv').concept_name)
feature_eicu = pd.read_csv('feature_eicu.csv')
feature_hicu = pd.read_csv('feature_hicu.csv')

## 매칭 안되는 부분
- drug : 세브란스에서 prescription은 쓸 수 없어서

## 매칭
- fuzzy algorithm 이용해서 비슷한 string에 대해서 매칭 진행하도록 한다.

In [82]:
feature_eicu.concept_table.unique()

array(['lab', 'adm'], dtype=object)

In [75]:
matching_field = feature_eicu[['concept_name']].merge(feature_hicu[['hicu_feature_english']], how='cross')

In [76]:
matching_field.head()

,concept_name,hicu_feature_english
0,-bands,address1
1,-bands,bp
2,-bands,Creatinine
3,-bands,body surface area
4,-bands,heightZ


In [77]:
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

def find_matching(eicu_feature, hicu_feature):
    ratio = fuzz.partial_ratio(eicu_feature.lower(), hicu_feature.lower())
    return ratio

In [78]:
matching_field['match_score'] = matching_field.apply(lambda x : find_matching(x['concept_name'], x['hicu_feature_english']), axis=1)

In [79]:
matching_field.query('match_score > 80').to_csv('matching.csv', index=False)

일일이 매칭 확인 후 맵핑된 것을 matching.csv로 업로드 하였음.

In [46]:
matched = pd.read_csv('matching.csv')

In [47]:
matched_result = matched[matched.matching == 1].reset_index(drop=True).eval('feature_new_name = hicu_feature_english.str.lower()')

# 이외에 height, weight, vital sign까지 포함해서 common feature로 이용

각 feature를 새로이 matching된 친구들로 대체하되, 그 정보를 feature_eicu, feature_hicu에 넣어준다.

In [48]:
feature_hicu.head()

,hicu_feature_original,hicu_feature_english
0,주소1,address1
1,혈압,bp
2,Creatinine,Creatinine
3,체표면적,body surface area
4,신장Z,heightZ


In [57]:
hicu_mapping = feature_hicu.merge(matched_result[['hicu_feature_english','feature_new_name']].drop_duplicates(), how='left')
hicu_mapping['feature_new_name'] = hicu_mapping.feature_new_name.fillna(hicu_mapping['hicu_feature_original'])

hicu_mapping.head()

,hicu_feature_original,hicu_feature_english,feature_new_name
0,주소1,address1,주소1
1,혈압,bp,혈압
2,Creatinine,Creatinine,creatinine
3,체표면적,body surface area,체표면적
4,신장Z,heightZ,신장Z


In [60]:
feature_eicu = feature_eicu.drop(columns = "Unnamed: 0")

In [70]:
eicu_mapping = feature_eicu.merge(matched_result[['concept_name','feature_new_name']], how='left')
eicu_mapping['feature_new_name'] = eicu_mapping.feature_new_name.fillna(eicu_mapping.concept_name)

eicu_mapping.head()

,concept_name,concept_table,MICU,SICU,gap,feature_new_name
0,-bands,lab,0.208736,0.147385,0.061351,-bands
1,-basos,lab,0.709761,0.634832,0.074929,-basos
2,-eos,lab,0.715908,0.638752,0.077156,-eos
3,-lymphs,lab,0.745945,0.658937,0.087008,-lymphs
4,-monos,lab,0.731073,0.645342,0.085732,-monos


save mapping.

In [74]:
hicu_mapping.to_csv("hicu_mapping.csv", index=False)
eicu_mapping.to_csv('eicu_mapping.csv', index=False)

In [75]:
eicu_mapping[eicu_mapping.concept_name.str]

,concept_name,concept_table,MICU,SICU,gap,feature_new_name
0,-bands,lab,0.208736,0.147385,0.061351,-bands
1,-basos,lab,0.709761,0.634832,0.074929,-basos
2,-eos,lab,0.715908,0.638752,0.077156,-eos
3,-lymphs,lab,0.745945,0.658937,0.087008,-lymphs
4,-monos,lab,0.731073,0.645342,0.085732,-monos
...,...,...,...,...,...,...
2341,uric acid,lab,0.035951,0.026191,0.009760,uric acid
2342,urinary creatinine,lab,0.072077,0.070481,0.001596,creatinine
2343,urinary osmolality,lab,0.048481,0.037534,0.010946,urinary osmolality
2344,urinary sodium,lab,0.100474,0.084828,0.015647,urinary sodium
